<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Inisiasi" data-toc-modified-id="Inisiasi-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Inisiasi</a></span></li><li><span><a href="#Pengujian-Model" data-toc-modified-id="Pengujian-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Pengujian Model</a></span></li><li><span><a href="#Kesimpulan" data-toc-modified-id="Kesimpulan-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Kesimpulan</a></span></li></ul></div>

## Introduction 

Pada penelitian ini, kita diminta oleh salah satu perusahaan telekomunikasi yang bergerak di bidang operator seluler bernama Megaline untuk mengembangkan sebuah model untuk merekomendasikan salah 1 dari 2 paket yang tersedia. Kita juga mendapat dataset yang berisi perilaku bulanan pada setiap pengguna. Model yang perlu kita kembangkan perlu setidaknya memiliki tingkat akurasi diatas 75%. Kita perlu menentukan jenis pendekatan yang akan digunakan, setelah itu, barulah kita bisa mencoba beberapa model untuk memilih kualitas model dengan akurasi tertinggi. 

## Inisiasi 

Seperti proses pada umumnya, kita perlu melakukan inisiasi terlebih dahulu sebelum memulai projek ini. Yaitu dengan mengimport semua library yang mungkin dibutuhkan dalam penelitian ini, dan membaca dataset yang sudah disediakan.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('datasets/users_behavior.csv')

In [3]:
#df.loc[(df['minutes'] > 500) | (df['messages'] > 50) | (df['mb_used'] > 15360), 'recommend_is_ultra'] = 1
#df.loc[(df['minutes'] <= 500) & (df['messages'] <= 50) & (df['mb_used'] <= 15360), 'recommend_is_ultra'] = 0
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Setelah itu, kita akan membagi dataset kita ke 3 dataset, yang terdiri dari data untuk training, data untuk melakukan validasi, dan data untuk testing. Pembagian ini dilakukan agar hasil penilaian kualitas model bisa diandalkan. Pembagiannya sediri dengan menggunakan rasio 3:1:1, yaitu 60% data training, 20% data validation dan 20% data test.

In [4]:
df_train, df_validation = train_test_split(df, test_size=0.40, random_state=12345)
df_validation, df_test = train_test_split(df_validation, test_size=0.50, random_state=12345)

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1928 entries, 3027 to 482
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [6]:
df_validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 1386 to 3197
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 643 entries, 160 to 2313
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [8]:
df.sample(20)

,calls,minutes,messages,mb_used,is_ultra
1129,71.0,453.98,43.0,13603.54,0
516,53.0,385.59,63.0,16987.84,0
1695,62.0,473.14,4.0,14623.29,1
2878,90.0,738.58,8.0,22070.76,0
173,64.0,403.51,54.0,19127.25,0
1697,33.0,214.38,14.0,11608.63,0
917,90.0,547.42,7.0,13741.10,0
2857,37.0,292.84,18.0,4080.59,0
292,86.0,567.58,47.0,16861.08,1
1376,122.0,848.33,0.0,17129.71,1


## Pengujian Model

Terakhir, setelah kita membagi dataset ke 3 bagian, kita bisa mulai melakukan pengujian model. Sebelumnya, kita perlu menganalisa terlebih dahulu terkait pendekatan yang akan digunakan untuk pengujian model ini. Karena dalam dataset kita terdapat kolom is_ultra dengan value 1 atau 0, maka bisa menggunakan pendekatan klasifikasi, karena target datanya berupa kategorikal. 

Dalam pendekatan kategorikal, ada 3 jenis model, yaitu Decision Tree, Random Forest, dan Regresi Logistik. Pada 2 model pertama ada hyperparameter yang perlu kita sesuaikan agar menentukan hasil terbaik. Untuk itu kita akan melakukan looping pada saat melakukan pengujian dengan model Decision Tree dan Random Forest.

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_validation = df_validation.drop(['is_ultra'], axis=1)
target_validation = df_validation['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [10]:
#decision tree
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_validation = model.predict(features_validation)
    print(f'depth: {depth}, accuracy: {accuracy_score(target_validation, predictions_validation)}')

depth: 1, accuracy: 0.7542768273716952
depth: 2, accuracy: 0.7822706065318819
depth: 3, accuracy: 0.7853810264385692
depth: 4, accuracy: 0.7791601866251944
depth: 5, accuracy: 0.7791601866251944


In [11]:
#random forest
for est in range(1,11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    print(f'n_estimator: {est}, accuracy: {model.score(features_validation, target_validation)}')

n_estimator: 1, accuracy: 0.7107309486780715
n_estimator: 2, accuracy: 0.7636080870917574
n_estimator: 3, accuracy: 0.7387247278382582
n_estimator: 4, accuracy: 0.7713841368584758
n_estimator: 5, accuracy: 0.749611197511664
n_estimator: 6, accuracy: 0.7807153965785381
n_estimator: 7, accuracy: 0.7682737169517885
n_estimator: 8, accuracy: 0.7822706065318819
n_estimator: 9, accuracy: 0.7729393468118196
n_estimator: 10, accuracy: 0.7853810264385692


In [12]:
#Logistic Regression
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
print(f'accuracy: {model.score(features_validation, target_validation)}')

accuracy: 0.7573872472783826


Dari hasil 3 pengujian model diatas, model Decision Tree dengan depth atau kedalaman pohon sebesar 3 merupakan akurasi tertinggi ketimbang semua model lain. Oleh karena itu, kita akan menggunakan model Decision tree dengan hyperparameter depth sebesar 3 untuk melakukan pengujian pada data test.

## Kesimpulan

In [13]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
print(f'accuracy: {accuracy_score(predictions_test, target_test)}')

accuracy: 0.7791601866251944


Dengan menggunakan model Decision tree dan max_depth 3, kita mendapatkan hasil akurasi sebesar 77%-78% saat melakukan pengujian dengan menggunakan test dataset. Nilai ini bisa dibilang acceptable karena ambang batas untuk tingkat akurasi pada penelitian ini adalah 75%. 